In [35]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
from sklearn.feature_extraction import stop_words
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.linear_model import LogisticRegression

### How do We Represent Language So Computers Can Analyze It?

If we want to analyze Reddit comments to make some prediction about them, we need to first find a way to **vectorize**, or numerically represent, the text of the comments.  This is the first concern of **natural language processing (NLP)**.  The entire comment collection taken together is referred to as the **corpus**, from the Latin for body.

One common approach to vectorizing words is called the **bag of words** representation.  In this strategy, we don't care about the relative positions of any of the words in a comment.  All we do is break the words into units (**tokenization**), count how often a given word appears in each comment, and then create a vector for each comment that is just the number of times each word appears, including all the 0s for words that appear somewhere in the corpus, but don't appear in that particular comment.  
When we do this, we usually want to exclude words that are extremely common, like "the", "and", etc... these words dominate frequencies, but their influence on the meaning of the text is usually minimal.  So it's better to leave them out if we're trying to make accurate predictions.  In NLP, these words are called **stop words**, and there are standard exclusion lists that are built into the available vectorizers as options.

There are a few different vectorizers to choose from:  **CountVectorizer()** and **TfidfVectorizer()** are the most widely used.  CountVectorizer tokenizes and counts, and that's it.  TfIdfVectorizer goes a step further, and **normalizes** the frequencies.  Basically, TfidfVectorizer will apply a transformation to our comment vectors that will **down-weight the influence of words that appear in a lot of comments, while up-weighting the influence of rarer words**.  The idea behind this is that common words tend to be less interesting to us; they give us less information about the comment.  Rarer words are more likely to have predictive value. 

That said, if a word is *too rare*, and only appears in one or two comments, there isn't much point to including it - what patterns can it really give us?  There are parameters that will allow us to only include words in our analysis if they appear in at least *n* comments.  

Single words may be able to help us predict whether a comment is AskMen or AskWomen, but what if we want to consider the frequency of certain *pairs* of words, or *triads*?  This is where **n-grams**, or groups of n words, come into play.  We can run our model with different combinations of n-grams included and see which performs best.  In this case, we found the best performance with a model that included n=1 (individual words), n=2 (pairs), and n=3.  Using n-grams gives us at least *some* characterization of the relative positions of words in each comment. It's no surprise that this improves our ability to predict a comment's origin.

### Logistic Regression

Predicting the subreddit that a comment has come from, or by extension the gender of a comment author, is a **classification problem**.  We want to classify each comment into one of two categories based on the combinations of words it contains.  One of the classic techniques in classification is **Logistic Regression**.  This name may be confusing, because typically when we talk about regression, we are trying to predict a continuous variable, rather than classifying into categories.  The connection is that logistic regression *does* technically generate a continuous variable.  But what it generates is a **predicted probability** value, which represents how likely the model thinks it is that the comment is in a given category.  Whichever category has the higher predicted probability will be the model's prediction for that comment.

You may be familiar with the ideas behind **linear regression**, where we try and map a line to data that minimizes the total prediction error across a bunch of observations.  Logistic regression is very similar.  It trains to minimize the error between it's predicted probabilities and the actual answers (0 or 1) for each comment.  

need to mention maximum likelihood estimation and logistic function, sigmoid function, maybe a picture...



I experimented with two other classification techniques as a part of this project, including **Naive Bayes** and sklearn's **Random Forest Classifier**.  These are entirely different algorithms which I will not discuss further here.  Random Forest, in particular, is a decision-tree based technique that tends to be very powerful.  However in this case, I found that ***Logistic Regression returned the highest accuracy scores***.  More importantly perhaps, logistic regression is the best suited to our goals later in this project, because it is extremely **interpretable**, meaning we can clearly characterize the factors (i.e.; the words) influencing its final predictions. These other methods are also interpretable to a degree, but less so.  This advantage is why logistic regression remains a *very* frequently used technique in the world of classification problems.

The last thing we need to know about logistic regression is that it involves **regularization**.  For a thorough explanation of regularization, please see the explanation I have made **[here](LINK THE EXPLANATION PARAGRAPH)**.  But the short version is that it is an extension of Logistic Regression's loss function that counterbalances the attempt to minimize error by also trying to minimize the scale of the feature coefficients.  Sometimes, especially when you have a large number of features like we do here (every word is a feature!), you can over-train to your learning data until you're just training to noise that will be useless for classifying new data.  By restricting the size of coefficients, regularization provides resistance against that happening.

### Finding the Best Model

We could talk for days about how to build the best performing model, and there are certainly other improvements we could explore.  But the core process is **GridSearching over hyperparameters**.  We talked above about some of the different hyperparameter options in language processing.  Here are some that we need to consider: 

- *Should we exclude stop words or not?*
- *Do we want to just use single words, or should we look at n-grams too?*
- *How many different comments does a word or n-gram need to appear in, for it to be included as a predictor?* 
- *What type of regularization do we want to use to prevent overfitting?*
- *How strong do we want our regularization to be?*

Basically, the GridSearch function will fit a model a few times each (splitting the data differently each time) across a bunch of different hyperparameter combinations, and select the hyperparameter combination that returns the highest average accuracy score.

In addition, we wanted to try out both vectorizers.  The vectorizer function and the LogisticRegression() function both have hyperparameters that we want to test with.  What we can do is set up a **pipeline**, which will let us run one GridSearch across the hyperparameters for both functions.  We'll do one pipeline for each vectorizer and see what gives the best result.

### Train Test Split

In [36]:
from sklearn.model_selection import train_test_split

#Separate data into comment text (features) and subreddit (target variable)
X = comments['body']
y = comments['subreddit']

#Train test split. Stratify=y guarantees that class balance will be maintained across train and test bloc
X_train, X_test, y_train, y_test = train_test_split(X,y,shuffle=True,stratify=y)

### GridSearch with CountVectorizer and Logistic Regression

In [42]:
pipe = Pipeline([
    ('vect', CountVectorizer()),
    ('model', LogisticRegression())
     ])

params = {
    'vect__ngram_range':[(1,3)],
    'vect__min_df':[2,5],
    'vect__stop_words':[None,'english'],
    'model__penalty':['l2','l1'],
    'model__C':[0.1, 1, 10],
}

gs_lr1 = GridSearchCV(pipe, params, cv=4, verbose=3, n_jobs=-1)

gs_lr1.fit(X_train, y_train)

Fitting 4 folds for each of 24 candidates, totalling 96 fits


[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:  3.9min
[Parallel(n_jobs=-1)]: Done  96 out of  96 | elapsed: 24.0min finished


GridSearchCV(cv=4, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...ty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'vect__ngram_range': [(1, 3)], 'vect__min_df': [2, 5], 'vect__stop_words': [None, 'english'], 'model__penalty': ['l2', 'l1'], 'model__C': [0.1, 1, 10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=3)

In [12]:
#Training data accuracy score
gs_lr1.score(X_train, y_train)

0.9105414233396021

In [13]:
#Test data accuracy score
gs_lr1.score(X_test, y_test)

0.7040798147158382

In [16]:
#Shows us which hyperparameters were chosen
gs_lr1.best_estimator_.steps

[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
          dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
          lowercase=True, max_df=1.0, max_features=None, min_df=2,
          ngram_range=(1, 2), preprocessor=None, stop_words=None,
          strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
          tokenizer=None, vocabulary=None)),
 ('model',
  LogisticRegression(C=0.1, class_weight=None, dual=False, fit_intercept=True,
            intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
            penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
            verbose=0, warm_start=False))]

### GridSearch with TdidfVectorizer and Logistic Regression

In [67]:
pipe2 = Pipeline([
    ('vect', TfidfVectorizer()),
    ('model', LogisticRegression())
     ])

params = {
    'vect__ngram_range':[(1,3)],
    'vect__min_df':[2,5],
    'vect__stop_words':[None,'english'],
    'model__penalty':['l2','l1'],
    'model__C':[0.1, 1, 10],
}

gs_lr2 = GridSearchCV(pipe2, params, cv=4, verbose=2, n_jobs=-1)

gs_lr2.fit(X_train, y_train)

Fitting 4 folds for each of 24 candidates, totalling 96 fits


[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:  4.6min
[Parallel(n_jobs=-1)]: Done  96 out of  96 | elapsed: 14.4min finished


GridSearchCV(cv=4, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
  ...ty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'vect__ngram_range': [(1, 3)], 'vect__min_df': [2, 5], 'vect__stop_words': [None, 'english'], 'model__penalty': ['l2', 'l1'], 'model__C': [0.1, 1, 10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=2)

In [68]:
#Training data score
gs_lr2.score(X_train, y_train)

0.8380481045234089

In [69]:
#Test data score
gs_lr2.score(X_test, y_test)

0.7051487618029574

In [70]:
#Shows which hyperparameters were chosen
gs_lr2.best_estimator_.steps

[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
          dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
          lowercase=True, max_df=1.0, max_features=None, min_df=5,
          ngram_range=(1, 3), norm='l2', preprocessor=None, smooth_idf=True,
          stop_words=None, strip_accents=None, sublinear_tf=False,
          token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
          vocabulary=None)),
 ('model',
  LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
            intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
            penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
            verbose=0, warm_start=False))]

### Model Results

The two vectorizers performed comparably, with accuracy around **70.5%**.  Whenever we attempt a classification problem, we want to keep in mind our **baseline accuracy**, which depends on the class balance of our target variable.  Think of it this way:  if we have 1000 comments, and 900 are from women, the model could simply predict that all comments are from women, and it would have a 90% accuracy score.  In fact, this is exactly what models will often do in such cases.  **This is why we try to balance our classes.**  In this case, we had 36335 AskWomen comments, 31019 AskMen, so the baseline accuracy = 36335/67354, or about **54%**.  Raising this to **70.5%** is a considerable improvement.

In evaluating your model's performance, it's also helpful to think about the reality of the challenge.  Predicting gender from text is not trivial. And the majority of these comments are short.  ***What percentage of Reddit comments do we think a human could correctly categorize as male or female, just looking at the text?***  70% seems fairly high. 

Let's proceed with the TfidfVec model, which has a slight edge in performance.  The final model:

- *uses Tfidf Vectorization with normalization*
- *uses Ridge regularization with strength $\alpha$ = 1*
- *includes n-grams of length 1, 2, and 3*
- *does *not* exclude stop words*
- *includes only words or n-grams that appear in at least 5 comments*

One thing that causes some concern is that the training scores are noticeably higher than the test scores, which typically indicates overfitting.  This is a bit surprising, because the cross-validation built in to the GridSearch process involves evaluating the model on unseen data.  To investigate this, I tried setting a maximum number of features and running the model again.  What I found is that if I maxed features at 3000, this effect disappeared, and returned in proportion as I raised the max up again.  Still, the model accuracy on the test set marginally improved with the increased features, so I will continue with the model as is.  

### Confusion Matrix for LogReg with Tfidf

In [76]:
def fancy_confusion_matrix(y_test, preds):

    cmat = confusion_matrix(y_test, preds)
    print(f'Accuracy: {accuracy_score(y_test, preds)}')
    print(classification_report(y_test, preds))
    return pd.DataFrame(cmat, columns=['Predicted ' + str(i) for i in ['AskMen','AskWomen']],\
            index=['Actual ' + str(i) for i in ['AskMen','AskWomen']])

predicts = gs_lr2.predict(X_test)
fancy_confusion_matrix(y_test, predicts)

Accuracy: 0.7051487618029574
             precision    recall  f1-score   support

     AskMen       0.70      0.63      0.66      7755
   AskWomen       0.71      0.77      0.74      9084

avg / total       0.70      0.71      0.70     16839



,Predicted AskMen,Predicted AskWomen
Actual AskMen,4879,2876
Actual AskWomen,2089,6995


In [26]:
predictions = pd.DataFrame(gs_lr2.predict_proba(X))
predictions['text'] = comments['body']

In [39]:
predictions.sort_values(1)[0:25]

,0,1,text
29958,0.995476,0.004524,You need to be able to distinguish between a g...
9195,0.990102,0.009898,Start with a physical change. Get a good hairc...
46357,0.989392,0.010608,Seyi shay. Her composure!
10730,0.988990,0.011010,She’s my wife :)
62159,0.987332,0.012668,My wife walked with her mom.
5575,0.987021,0.012979,"Well, most likely if she’s sucking on yo dick ..."
6819,0.986445,0.013555,&gt;girl &gt;friend Pick one OP
23865,0.985701,0.014299,"Usually, you are attracted to the person befor..."
26589,0.985633,0.014367,You tried... She declined... She knows you tri...
20508,0.985479,0.014521,"Say nothing, don't answer any questions. Becau..."


In [71]:
coefs = pd.DataFrame(gs_lr2.best_estimator_.steps[1][1].coef_)
coefs.columns=gs_lr2.best_estimator_.steps[0][1].get_feature_names()
coefs_ranks = 
coef_ranks.to_csv('coefs_ranks')

Close examination of the coefficients in the Tfidf model above show coefficients that are reasonably sized, and also very plausible as predictors.  So the models do seem to be working correctly.

### Modeling Only For Long Comments (>20 Words)

The last thing I am interested in here is how this will perform if I limit the comments to word length > 20.  It seems likely that the model will be able to be more accurate for full-length comments.

In [39]:
longcomments = comments[comments['word_length']>=20].copy()
len(longcomments)

45194

In [35]:
X_long = longcomments['body']
y_long = longcomments['subreddit']
X_train_long, X_test_long, y_train_long, y_test_long = train_test_split(X_long,y_long,shuffle=True,stratify=y_long)

In [36]:
pipelong = Pipeline([
    ('vect', TfidfVectorizer()),
    ('model', LogisticRegression())
     ])

params = {
    'vect__ngram_range':[(1,3)],
    'vect__min_df':[2,5],
    'vect__stop_words':[None,'english'],
    'model__penalty':['l2','l1'],
    'model__C':[0.1, 1, 10],
}

gs_lr_long = GridSearchCV(pipelong, params, cv=4, verbose=2, n_jobs=-1)

gs_lr_long.fit(X_train_long, y_train_long)

Fitting 4 folds for each of 24 candidates, totalling 96 fits


[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:  4.7min
[Parallel(n_jobs=-1)]: Done  96 out of  96 | elapsed: 13.9min finished


GridSearchCV(cv=4, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
  ...ty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'vect__ngram_range': [(1, 3)], 'vect__min_df': [2, 5], 'vect__stop_words': [None, 'english'], 'model__penalty': ['l2', 'l1'], 'model__C': [0.1, 1, 10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=2)

In [37]:
gs_lr_long.score(X_train_long,y_train_long)

0.998465850420416

In [38]:
gs_lr_long.score(X_test_long,y_test_long)

0.7258164439330914

There is a small improvement in predictive power, but for the most part the numbers mimic those seen in the wider set.

### Conclusions, Limitations

In [ ]:
Limitations of AskMen vs AskWomen first-tier replies as a proxy for gender:  "I'm a girl"

In [ ]:
Would be cool to look at AskMen comments and see which it most thinks are from a woman, and vice versa.